In [1]:
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os
load_dotenv()

URI = "neo4j+s://e5703e68.databases.neo4j.io"
AUTH = (os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))

In [6]:
driver = GraphDatabase.driver(URI, auth=AUTH)
print(driver.verify_authentication())
print(driver.verify_connectivity())

True
None


In [7]:
summary = driver.execute_query("""
    CREATE (a:Person {name: $name})
    CREATE (b:Person {name: $friendName})
    CREATE (a)-[:KNOWS]->(b)
    """,
    name="Alice", friendName="David",
    database_="e5703e68",
).summary
print("Created {nodes_created} nodes in {time} ms.".format(
    nodes_created=summary.counters.nodes_created,
    time=summary.result_available_after
))

Created 2 nodes in 1 ms.


In [12]:
records, summary, keys = driver.execute_query("""
    MATCH (p:Person)-[:KNOWS]->(k:Person)
    RETURN p.name AS name, k.name as know_name
    """,
    database_="e5703e68",
)

# Loop through results and do something with them
for record in records:
    print(record.data())  # obtain record as dict

# Summary information
print("The query `{query}` returned {records_count} records in {time} ms.".format(
    query=summary.query, records_count=len(records),
    time=summary.result_available_after
))

{'name': 'Alice', 'know_name': 'David'}
The query `
    MATCH (p:Person)-[:KNOWS]->(k:Person)
    RETURN p.name AS name, k.name as know_name
    ` returned 1 records in 1 ms.
